# Optimization Code

In this sandbox, I optimize the prompts for the LLM chain using Dspy


## Settings

This import all the packages, separate functions and sets the LLM to the chain defined in the main module.

In [5]:
import groq
import os
from dotenv import load_dotenv
import dspy
import numpy as np
from dspy.evaluate.metrics import answer_exact_match
import pandas as pd
from IPython.core.display import Markdown
from dspy import Example
from dspy.teleprompt import BootstrapFewShot
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.evaluate import Evaluate
from module_v002 import FullLLMChain
from optimize import passage_similarity_metric, custom_evaluation_function, similar_score_metric, evaluate_expectations_metric
from data.preprocess import create_dspy_examples_train_test_validation_sets
import json
import pandas as pd

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")


llama3_8b = dspy.OllamaLocal(model = "llama3:8b",
                             temperature = 0,
                             max_tokens = 800)

gpt35turbo = dspy.OpenAI(model = "gpt-3.5-turbo-0125",
                         api_key = openai_api_key,
                         temperature = 0,
                         max_tokens = 800,
                         model_type = "chat")    


# Preprocess Data

In [6]:
data = pd.read_excel("data/300_snippets_transcripts_all_labeled_v002.xlsx")

In [7]:
train_set, test_set, validation_set = create_dspy_examples_train_test_validation_sets(
    data=data, 
    train_size=50, 
    test_size=50,
    validation_size=50
)

## Optimization

This optimizes the prompts and saves the optimized LLM as well as the last 10 instances prompted to the LLM

In [8]:
dspy.settings.configure(lm=gpt35turbo)
full_llm_chain = FullLLMChain()

In [9]:
config = dict(max_bootstrapped_demos=2, max_labeled_demos=1, max_rounds=1, max_errors=1, num_candidate_programs = 4)
teleprompter = BootstrapFewShotWithRandomSearch(metric=evaluate_expectations_metric, **config)
optimized_llm = teleprompter.compile(full_llm_chain, trainset=train_set, valset=test_set)

Going to sample between 1 and 2 traces per predictor.
Will attempt to train 4 candidate sets.


Average Metric: 22.0 / 50  (44.0): 100%|██████████| 50/50 [00:24<00:00,  2.03it/s]


Average Metric: 22.0 / 50  (44.0%)
Score: 44.0 for set: [0, 0, 0]
New best score: 44.0 for seed -3
Scores so far: [44.0]
Best score: 44.0


Average Metric: 21.0 / 47  (44.7):  94%|█████████▍| 47/50 [00:24<00:02,  1.36it/s]

Backing off 0.7 seconds after 1 tries calling function <function GPT3.request at 0x10a33ed40> with kwargs {'temperature': 0.7000861588493462}


...

Backing off 28.4 seconds after 6 tries calling function <function GPT3.request at 0x10a33ed40> with kwargs {'temperature': 0.7000656664557154}


Average Metric: 27.5 / 44  (62.5):  88%|████████▊ | 44/50 [00:52<00:06,  1.14s/it]

Backing off 2.8 seconds after 3 tries calling function <function GPT3.request at 0x10a33ed40> with kwargs {'temperature': 0.7000124110942705}


Average Metric: 30.0 / 50  (60.0): 100%|██████████| 50/50 [01:22<00:00,  1.65s/it]

Average Metric: 30.0 / 50  (60.0%)
Score: 60.0 for set: [1, 1, 1]
New best score: 60.0 for seed 3
Scores so far: [44.0, 46.0, 53.0, 43.0, 40.0, 48.0, 60.0]
Best score: 60.0
Average of max per entry across top 1 scores: 0.6
Average of max per entry across top 2 scores: 0.78
Average of max per entry across top 3 scores: 0.9
Average of max per entry across top 5 scores: 0.93
Average of max per entry across top 8 scores: 0.96
Average of max per entry across top 9999 scores: 0.96
7 candidate programs found.


In [10]:
optimized_llm.save("optimized_llm_chains/optimized_gpt_chain_v010.json")

In [11]:
llama3_8b.inspect_history(n=1)

In [12]:
gpt35turbo.inspect_history(n=2)





---CONTEXT---
    You are an experienced financial analyst known for your expertise in evaluating and interpreting expectations related to the financial stability and solvency of countries.

    ---TASK---
    Please assess the expectation towards the solvency of the country mentioned in the given text excerpt. This includes evaluating the country's financial stability and ability to meet its obligations.    
    
    ---GUIDELINES---
    - Use the following scale for your assessment: -2 = very negative, -1 = somewhat negative, 0 = neutral, 1 = somewhat positive, 2 = very positive

---

Follow the following format.

Country Keyword: keyword that represents a country

Country Role: role of the country in the excerpt of a financial services company's earnings call transcript

Excerpt: excerpt from a financial services company's earnings conference call

Reasoning: Let's think step by step in order to ${produce the answer}. We ...

Answer: one of: [-2, -1, 0, 1, 2]. Only respond wiht 

## Custom Evaluation

Here, I just manually inspect what happens to control the workflow.

In [13]:
custom_evaluation_function(validation_set=valset, 
                           llm= full_llm_chain, 
                           metric_for_evaluation="evaluate_expectations", 
                           show_examples=2)

NameError: name 'valset' is not defined

In [ ]:
gpt35turbo.inspect_history(n = 2)

In [ ]:
evaluate = Evaluate(devset=valset, metric= evaluate_expectations_metric, num_threads=4, display_progress=True, display_table=4)